In [1]:
!pip install scikit-image scikit-learn matplotlib pandas tqdm gco-wrapper line_profiler torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 103.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 97.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 107.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 104.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.0/243.0 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.8/714.8 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.9 MB/s eta 0:00:0

In [3]:
!wget -P datasets https://storage.googleapis.com/thesis_cloud_files/CocoStuff_curated_coarse_val_only.zip
!apt-get update
!apt-get install -y unzip
!unzip datasets/CocoStuff_curated_coarse_val_only.zip -d datasets

--2024-02-19 01:08:05--  https://storage.googleapis.com/thesis_cloud_files/CocoStuff_curated_coarse_val_only.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.36.155, 142.251.37.123, 142.251.36.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.36.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 378408755 (361M) [application/zip]
Saving to: ‘datasets/CocoStuff_curated_coarse_val_only.zip’

CocoStuff_curated_c 100%[===================>] 360.88M  37.1MB/s    in 10s     

2024-02-19 01:08:15 (35.2 MB/s) - ‘datasets/CocoStuff_curated_coarse_val_only.zip’ saved [378408755/378408755]



In [1]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
from GraPL import GraPL_Segmentor, side_by_side, view_multichannel, PatchDL, GraPLNet, segment#, segment_coco_val
import glob
import tqdm
import matplotlib.pyplot as plt
from GraPL.evaluate import *
import numpy as np
import warnings
import torch
import pandas as pd
import os
import json
warnings.filterwarnings("ignore")

env: CUDA_LAUNCH_BLOCKING=1
env: CUBLAS_WORKSPACE_CONFIG=:4096:8


/usr/local/lib/python3.10/dist-packages/gco/__init__.py:7: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
  if not hasattr(numpy, name):


In [2]:
val_image_ids = [os.path.basename(f) for f in glob.glob('datasets/CocoStuff_curated_coarse_val_only/images/val2017/*')]
val_image_ids = [f.split('.')[0] for f in val_image_ids]

params = {
    'iterations': 4,
    'k': 14,
    'd': 32,
    'lambda_': 64,
    'subset_size': 0.5,
    'max_epochs': 40,
    'min_epochs': 12,
    'n_filters': 32,
    'bottleneck_dim': 8,
    'compactness': 0.1,
    'sigma': 10,
    'seed': 0,
    'use_continuity_loss': True,
    'continuity_range': 1,
    'continuity_p': 1,
    'continuity_weight': 2,
    'use_min_loss': True,
    'use_coords': False,
    'use_embeddings': False,
    'use_color_distance_weights': True,
    'initialization_method': 'slic',
    'use_fully_connected': True,
    'use_collapse_penalty': False,
    'use_cold_start': False,
    'num_layers': 3,
    'use_graph_cut': True
}

In [3]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

def segment_coco_val(results_dir=None, resume=False, progress_bar=None, debug_num=-1, should_score=True, **hyperparams):
    if results_dir is None:
        raise ValueError("results_dir must be specified")
    os.makedirs(results_dir, exist_ok=True)
    val_image_ids = [os.path.basename(f) for f in glob.glob('datasets/CocoStuff_curated_coarse_val_only/images/val2017/*')]
    val_image_ids = [f.split('.')[0] for f in val_image_ids]
    image_paths = [f"datasets/CocoStuff_curated_coarse_val_only/images/val2017/{id}.jpg" for id in val_image_ids]
    paramset_scores = {}
    if progress_bar is None:
        progress_bar = tqdm.tqdm(total=len(image_paths))
    if debug_num > 0:
        image_paths = image_paths[:debug_num]

    def per_image_mapable(image_path):
        id = image_path.split('/')[-1].split('.')[0]
        image = plt.imread(image_path)[:,:,:3]
        image_scores = {}
        save_path = f'{results_dir}/{id}.png'
        if resume and os.path.exists(save_path):
            if should_score:
                image_scores = coco_score(id, save_path)
                paramset_scores[id] = image_scores
            progress_bar.update(1)
            return
        seg = segment(image, save_path=save_path, **hyperparams)
        if should_score:
            image_scores = coco_score(id, save_path)
            paramset_scores[id] = image_scores
        progress_bar.update(1)

    with ThreadPoolExecutor(max_workers=8) as executor:
        tqdm.tqdm(executor.map(per_image_mapable, image_paths), total=len(image_paths))


    # for image_path in image_paths:
    #     id = image_path.split('/')[-1].split('.')[0]
    #     image = plt.imread(image_path)[:,:,:3]
    #     image_scores = {}
    #     save_path = f'{results_dir}/{id}.png'
    #     if resume and os.path.exists(save_path):
    #         if should_score:
    #             image_scores = coco_score(id, save_path)
    #             paramset_scores[id] = image_scores
    #         progress_bar.update(1)
    #         continue
    #     seg = segment(image, save_path=save_path, **hyperparams)
    #     if should_score:
    #         image_scores = coco_score(id, save_path)
    #         paramset_scores[id] = image_scores
    #     progress_bar.update(1)
    with open(f'{results_dir}/scores.json', 'w') as fp:
        results = {"hyperparams": hyperparams, "scores": paramset_scores}
        json.dump(results, fp)
    return paramset_scores

In [5]:
def run_experiment(base_params, num_trials=1, should_score=True, seeds=None):
    results_dir = f'experiment_results/coco_val'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    aggregate_scores = {}
    trials = []
    mean_over_trials = {}
    if seeds is None:
        seeds = list(range(num_trials))
    print(f'Running experiment with seeds: {seeds}')
    with tqdm.tqdm(total=len(seeds) * len(val_image_ids)) as progress_bar:
        for trial_num in seeds:
            params = base_params.copy()
            params['seed'] = trial_num
            trial_results_dir = f'{results_dir}/{trial_num}'
            trial_scores = segment_coco_val(results_dir=trial_results_dir, debug_num=-1, resume=True, progress_bar=progress_bar, should_score=should_score, **params)
            image_ids = list(trial_scores.keys())
            metrics = trial_scores[image_ids[0]].keys()
            trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
            trials.append(trial_scores)
    for metric in trials[0]:
        mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
    aggregate_scores = mean_over_trials
    return aggregate_scores

run_experiment(params, num_trials=1, should_score=True)

Running experiment with seeds: [0]


  0%|          | 0/2175 [00:00<?, ?it/s]


  0%|          | 0/2175 [00:01<?, ?it/s]


IndexError: list index out of range

In [11]:
import json
import numpy as np 
import glob
import os

experiment_name = "naive_voc"
results_dir = f'experiment_results/{experiment_name}'
results_trial_dirs = [d for d in glob.glob(f'{results_dir}/*') if os.path.isdir(d)]
all_object_jaccards = []
all_class_jaccards = []
for trial_dir in results_trial_dirs:
    trial_scores = json.load(open(f'{trial_dir}/scores.json'))
    for image_id in trial_scores["scores"].keys():
        all_object_jaccards.append(trial_scores["scores"][image_id]['jaccard_object'])
        all_class_jaccards.append(trial_scores["scores"][image_id]['jaccard_class'])

all_object_jaccards = [(x if not np.isnan(x) else 0) for x in all_object_jaccards]
all_class_jaccards = [(x if not np.isnan(x) else 0) for x in all_class_jaccards]
print(f"Mean Object Jaccard: {np.mean(all_object_jaccards)}")
print(f"Mean Class Jaccard: {np.mean(all_class_jaccards)}")

Mean Object Jaccard: 0.5415332208786632
Mean Class Jaccard: 0.5530006427366786
